In [9]:
# open file 
mni = open("input.txt","r");
hex_bytes = readline(mni)
bits_array = bitstring.([parse(UInt16,hex_bytes[i], base=16) for i in 1:length(hex_bytes)])
bits = ""
for b in bits_array
    tail = b[end-3:end]
    bits *= tail
end
@show hex_bytes
@show bits; # just fix the last 2 bytes

hex_bytes = "A20D6CE8F00033925A95338B6549C0149E3398DE75817200992531E25F005A18C8C8C0001849FDD43629C293004B001059363936796973BF3699CFF4C6C0068C9D72A1231C339802519F001029C2B9C29700B2573962930298B6B524893ABCCEC2BCD681CC010D005E104EFC7246F5EE7328C22C8400424C2538039239F720E3339940263A98029600A80021B1FE34C69100760B41C86D290A8E180256009C9639896A66533E459148200D5AC0149D4E9AACEF0F66B42696194031F000BCE7002D80A8D60277DC00B20227C807E8001CE0C00A7002DC00F300208044E000E69C00B000974C00C1003DC0089B90C1006F5E009CFC87E7E43F3FBADE77BE14C8032C9350D005662754F9BDFA32D881004B12B1964D7000B689B03254564414C016B004A6D3A6BD0DC61E2C95C6E798EA8A4600B5006EC0008542D8690B80010D89F1461B4F535296B6B305A7A4264029580021D1122146900043A0EC7884200085C598CF064C0129CFD8868024592FEE9D7692FEE9D735009E6BBECE0826842730CD250EEA49AA00C4F4B9C9D36D925195A52C4C362EB8043359AE221733DB4B14D9DCE6636ECE48132E040182D802F30AF22F131087EDD9A20804D27BEFF3FD16C8F53A5B599F4866A78D7898C0139418D00424EBB459915200C0BC01098B527C99F4EB54CF0450014A95863BDD3

In [30]:
function VVV(bits, c)
    return parse(Int, bits[c:c+2], base=2)
end
function TTT(bits, c)
    return parse(Int, bits[c+3:c+5], base=2)
end
function I(bits, c)
    return parse(Bool, bits[c+6]) ? "Packets" : "Bits"
end
function LLLL(bits, c)
    if TTT(bits, c) == 4
        return -1
    end
    
    if I(bits, c) == "Packets" 
        return parse(Int, bits[c+7:c+7+10], base=2)
    else
        return parse(Int, bits[c+7:c+7+14], base=2)
    end
end
function A_idx(bits, c)
    if I(bits, c) == "Packets" 
        return c+18
    else
        return c+22
    end
end
function get_node_literal(bits, c)
    idx = c+6
    bitstr = ""
    while bits[idx] == '1'
        bitstr *= bits[idx+1:idx+4]
        idx += 5
    end
    bitstr *= bits[idx+1:idx+4]
    idx += 5
    return parse(Int, bitstr, base=2), idx
end

function get_next_idx(bits, c)
    if TTT(bits, c) == 4
        val, next = get_node_literal(bits,c)
        return next
    else
        if I(bits, c) == "Packets" 
            tot_len = A_idx(bits, c)
            for sn in get_subnodes(bits, c)
                tot_len += get_next_idx(bits, sn)
            end
            return tot_len  
        else
            return LLLL(bits, c) + A_idx(bits, c)
        end
    end
end
function get_subnodes(bits, c)
    subnodes = Array{Int64}(undef, 0)
    ll = TTT(bits, c) != 4 ? LLLL(bits,c) : get_node_literal(bits,c)
    sn = A_idx(bits, c)
    #@show c, VVV(bits, c), TTT(bits, c), I(bits, c), ll, sn 
    if I(bits, c) == "Packets"
        child_count = 0
        while child_count < LLLL(bits, c) && length(bits)-sn  > 6
            #@show sn, TTT(bits, sn)
            temp_nodes = []
            if TTT(bits, sn) == 4
                push!(subnodes, sn)
                push!(temp_nodes, sn)
            else
                temp_nodes = copy(get_subnodes(bits, sn))
                push!(subnodes, sn)
                
                #@show "Before $sn"
                #for n in get_subnodes(bits, sn)
                #    push!(subnodes,n)
                #end
                #@show "After $sn"
                #vcat(subnodes, temp_nodes)
                #@show subnodes
            end  
            #@show "Packets"
            #@show subnodes
            sn = get_next_idx(bits, temp_nodes[end]) #problem here
            child_count += 1
        end
    elseif I(bits, c) == "Bits"
        while sn < A_idx(bits, c) + LLLL(bits, c) && length(bits)-sn  > 6
            #@show sn, TTT(bits, sn)
            temp_nodes = []
            if TTT(bits, sn) == 4
                push!(subnodes, sn)
                push!(temp_nodes, sn)
            else
                temp_nodes = copy(get_subnodes(bits, sn))
                push!(subnodes, sn)
                #for n in get_subnodes(bits, sn)
                #    push!(subnodes,n)
                #end
                #@show "After $sn"
                #@show subnodes
                #vcat(subnodes, temp_nodes)
            end
            #@show "Bits"
            #@show subnodes
            sn = get_next_idx(bits, temp_nodes[end]) 
        end
    end
       
    #@show "Return subnodes from parent node $c"
    #@show subnodes
    return subnodes
end
function initiailize_node(bits,c)
    V = VVV(bits, c)
    T = TTT(bits, c)
    if T == 4
        Literal, next = get_node_literal(bits, c)
        return (c = c, V = V, T = T, Literal = Literal, next = next)
    else
        L = LLLL(bits, c)
        L_bit_len = I(bits, c) == "Packets" ? 11 : 15
        return (c = c, V = V, T = T, I = I(bits, c), L = L, subnodes = [], next = 8 + L_bit_len)
    end
end
function get_subnodes(bits,a_idx)
   cn = initiailize_node(bits, a_idx)
   
    if cn.I == "Packets"
        while length(subnodes) <
    else
    end
end

function get_node(bits,c)
    V = VVV(bits, c)
    T = TTT(bits, c)
    @show c, V, T
    if T == 4
        Literal, next = get_node_literal(bits, c)
        return c, V, T, Literal, next
    else
        L = LLLL(bits, c)
       
        snl = get_subnodes(bits, c)
        return c, V, T, I(bits, c), L, snl
    end
end
function get_value_sum(bits,c)
    if TTT(bits, c) == 4
        return VVV(bits, c)
    else
        sum = VVV(bits, c)
        for sn in get_subnodes(bits, c)
            sum += get_value_sum(bits,sn)
        end
        return sum
    end
end

get_value_sum (generic function with 1 method)

In [16]:

#packet_pointers = get_subnodes(bits, 1)

In [ ]:
a = get_node(bits,1)
#@show get_node(bits,23);
#@show get_node(bits,41);
#@show get_node(bits,59);
#@show get_node(bits,70);
#@show get_node(bits,81);
#@show get_node(bits,92);
#@show get_node(bits,103);
#@show get_node(bits,107);
#@show bits[23:25], bits[26:28], bits[29:33]
#@show bits[34:36], bits[37:39], bits[40:44], bits[45:49]

In [5]:
get_value_sum(bits, 1)

31

In [10]:
LLLL(bits,1)

53

In [6]:
test_node = 1
@show VVV(bits, test_node)
@show a= TTT(bits, test_node)

if a == 4
    @show get_node_literal(bits, test_node)
    @show get_next_idx(bits, test_node)
else
    @show I(bits, test_node);
    @show LLLL(bits, test_node);
    @show A_idx(bits, test_node);
end

#get_next_idx(bits, test_node);

VVV(bits, test_node) = 5
a = TTT(bits, test_node) = 0
I(bits, test_node) = "Packets"
LLLL(bits, test_node) = 53
A_idx(bits, test_node) = 19


In [7]:
test_node = 19
@show VVV(bits, test_node)
@show a= TTT(bits, test_node)

if a == 4
    @show get_node_literal(bits, test_node)
    @show get_next_idx(bits, test_node)
else
    @show I(bits, test_node);
    @show LLLL(bits, test_node);
    @show A_idx(bits, test_node);
end

#get_next_idx(bits, test_node);

VVV(bits, test_node) = 5
a = TTT(bits, test_node) = 4
get_node_literal(bits, test_node) = (211, 35)
get_next_idx(bits, test_node) = 35


In [22]:
bits[1:18]

"101000100000110101"

In [23]:
bits[19:36]

"101100111010001111"

In [147]:
test_node = 35
@show VVV(bits, test_node)
@show TTT(bits, test_node)
    @show I(bits, test_node);
    @show LLLL(bits, test_node);
    @show A_idx(bits, test_node);


VVV(bits, test_node) = 6
TTT(bits, test_node) = 0
I(bits, test_node) = "Bits"
LLLL(bits, test_node) = 51
A_idx(bits, test_node) = 57


(c = 19, V = 5, T = 4, Literal = 211, next = 35)

In [35]:
bits[19:35]

"10110011101000111"

In [84]:
bits2 = bits

"101000100000110101101100111010001111000000000000001100111001001001011010100101010011001110001011011001010100100111000000000101001001111000110011100110001101111001110101100000010111001000000000100110010010010100110001111000100101111100000000010110100001100011001000110010" ⋯ 5052 bytes ⋯ "110000100000000100100000001110000000001101010011000000000000010000111010000010111100010000001000110110000010000000001110110001111110100010000101000110110110010110001111000011000000001011101001101001011010100110100101001010010100000001111001011000100110100110110010000000"

In [140]:
function get_node2(b)
    @show b[1:30]
    V = parse(Int, b[1:3], base=2)
    b = b[4:end]
    T = parse(Int, b[1:3], base=2)
    b = b[4:end]
    if T == 4
        @show b[1:5]
        @show lit_str = b[2:5]
        while b[1] == '1'
            @show lit_str *= b[2:5]
            b = b[6:end]
            @show b[1:5]
        end
        val = parse(Int, lit_str, base=2)
        @show (V, T, val), b[1:7] 
        return (V, T, val), b 
    else
        I = b[1]
        b = b[2:end]
        sp = []
        if I == "0"
            L = parse(Int, b[1:15], base=2)
            b = b[16:end]
            d = b[1:L]
            while length(d) > 0
                node, d = get_node2(d)
                push!(sp, node)
                @show sp, d
            end
        else
            L = parse(Int, b[1:11], base=2)
            b = b[12:end]
            for i in 1:L
                node, b = get_node2(b)
                push!(sp, node)
                @show sp
            end
        end
        @show (V, T, I, L, sp), b[1:7]
        return (V, T, I, L, sp), b
    end
end

get_node2 (generic function with 1 method)

In [141]:
function get_total_vals(packet)
    tot = 0
    tot += packet[1]
    
    if packet[2] != 4
        tot += sum(sp->get_total_vals(sp),packet[5])
    end
    return tot
end

get_total_vals (generic function with 1 method)

In [142]:
bits2 = bits
a, b = get_node2(bits2)

b[1:30] = "101000100000110101101100111010"
b[1:30] = "101100111010001111000000000000"
b[1:5] = "11101"
lit_str = b[2:5] = "1101"
lit_str *= b[2:5] = "11011101"
b[1:5] = "00011"
((V, T, val), b[1:7]) = ((5, 4, 221), "0001111")
sp = Any[(5, 4, 221)]
b[1:30] = "000111100000000000000110011100"
((V, T, I, L, sp), b[1:7]) = ((0, 7, '1', 0, Any[]), "0001100")
sp = Any[(5, 4, 221), (0, 7, '1', 0, Any[])]
b[1:30] = "000110011100100100101101010010"
b[1:30] = "101101010010101001100111000101"
b[1:30] = "100111000101101100101010010011"
b[1:30] = "101010010011100000000010100100"
b[1:30] = "000010100100111100011001110011"
b[1:30] = "011001110011000110111100111010"
b[1:30] = "111100111010110000001011100100"
b[1:5] = "11101"
lit_str = b[2:5] = "1101"
lit_str *= b[2:5] = "11011101"
b[1:5] = "01100"
((V, T, val), b[1:7]) = ((7, 4, 221), "0110000")
sp = Any[(7, 4, 221)]
b[1:30] = "011000000101110010000000001001"
b[1:30] = "000000001001100100100101001100"
b[1:30] = "100101001100011110001001011111"
b[1:30] 

sp = Any[(1, 3, '1', 0, Any[]), (0, 4, 6)]
b[1:30] = "001101100010011111000101000110"
b[1:30] = "000101000110000110110100111101"
b[1:30] = "110100111101010011010100101001"
b[1:5] = "11110"
lit_str = b[2:5] = "1110"
lit_str *= b[2:5] = "11101110"
b[1:5] = "10100"
lit_str *= b[2:5] = "111011100100"
b[1:5] = "11010"
lit_str *= b[2:5] = "1110111001001010"
b[1:5] = "10010"
lit_str *= b[2:5] = "11101110010010100010"
b[1:5] = "10010"
lit_str *= b[2:5] = "111011100100101000100010"
b[1:5] = "11010"
lit_str *= b[2:5] = "1110111001001010001000101010"
b[1:5] = "11011"
lit_str *= b[2:5] = "11101110010010100010001010101011"
b[1:5] = "01011"
((V, T, val), b[1:7]) = ((6, 4, 3997835947), "0101100")
sp = Any[(6, 4, 3997835947)]
b[1:30] = "010110011000001011010011110100"
b[1:30] = "010011110100100001001100100000"
b[1:30] = "001100100000000101001010110000"
b[1:5] = "10000"
lit_str = b[2:5] = "0000"
lit_str *= b[2:5] = "00000000"
b[1:5] = "00001"
((V, T, val), b[1:7]) = ((1, 4, 0), "0000101")
sp = Any[(1, 

b[1:30] = "000000000001000010101000011101"
b[1:30] = "101000011101010000011001100000"
b[1:30] = "011001100000111100000000000000"
b[1:30] = "000000000000100001110100010010"
b[1:30] = "110100010010011000111000010000"
b[1:5] = "01001"
lit_str = b[2:5] = "1001"
((V, T, val), b[1:7]) = ((6, 4, 9), "0100100")
sp = Any[(6, 4, 9)]
b[1:30] = "010010011000111000010000011011"
b[1:30] = "010000011011100111101001011111"
b[1:30] = "101001011111111110001001100000"
b[1:30] = "001001100000000000010100100001"
((V, T, I, L, sp), b[1:7]) = ((1, 1, '1', 0, Any[]), "0101001")
sp = Any[(1, 1, '1', 0, Any[])]
b[1:30] = "010100100001001000111000100011"
b[1:5] = "10000"
lit_str = b[2:5] = "0000"
lit_str *= b[2:5] = "00000000"
b[1:5] = "10010"
lit_str *= b[2:5] = "000000000010"
b[1:5] = "00111"
((V, T, val), b[1:7]) = ((2, 4, 2), "0011100")
sp = Any[(1, 1, '1', 0, Any[]), (2, 4, 2)]
b[1:30] = "001110001000111000100111110110"
b[1:30] = "100111110110011001010011001100"
b[1:30] = "010011001100111101110001100110"
b[

LoadError: BoundsError: attempt to access 24-codeunit String at index [1:30]

In [124]:
test_node = 35
@show VVV(bits, test_node)
@show TTT(bits, test_node)
@show get_node_literal(bits, test_node)
    @show I(bits, test_node);
    @show LLLL(bits, test_node);
    @show A_idx(bits, test_node);


VVV(bits, test_node) = 6
TTT(bits, test_node) = 0
get_node_literal(bits, test_node) = (0, 46)
I(bits, test_node) = "Bits"
LLLL(bits, test_node) = 51
A_idx(bits, test_node) = 57


In [120]:
bits[1:34], bits[35:end]

("1010001000001101011011001110100011", "110000000000000011001110010010010110101001010100110011100010110110010101001001110000000001010010011110001100111001100011011110011101011000000101110010000000001001100100100101001100011110001001011111000000000101101000011000110010001100100011000000000000000001100001001001111111011101010000110110001010011100001010010011000000000100101100000000000100000101100100110110001110010011011001111001011010010111001110111111001101101001100111001111111101001100011011000000000001101000110010011101011100101010000100100011000111000011001110011000000000100101000110011111000000000001000000101001110000101011100111000010100101110000000010110010010101110011100101100010100100110000001010011000101101101011010100100100100010010011101010111100110011101100001010111100110101101000000111001100000000010000110100000000010111100001000001001110111111000111001001000110111101011110111001110011001010001100001000101100100001000000000001000010010011000010010100111000000000111001001000

In [77]:
@show b = a[1:3]
@show a = a[4:end]

b = a[1:3] = "123"
a = a[4:end] = "45"


"45"